In [1]:
from datetime import date
from src.market import Market
from src.portfolio import Portfolio
from src.analysis import Analysis, Benchmark, Metric
from src.rebalance import Rebalance
from src.strategy import StopGainAndLoss
from src.backtest import BackTest
from src.factor.sales_growth import SalesGrowthFactor
from src.factor.const import SECTOR_ETF

In [2]:
start_date = date.fromisoformat("2022-01-01")
end_date = date.fromisoformat("2023-10-21")
security_universe = SECTOR_ETF

market = Market(security_universe)

In [3]:
### Long factor
long_factor = SalesGrowthFactor(security_universe, start_date, end_date, "long", 12)
long_position = long_factor.get_position(start_date)
long_portfolio = Portfolio(100.0, start_date, end_date)
long_factor.set_portfolio_at_start(long_portfolio, long_position)

blacklist = []
strategy = StopGainAndLoss(long_portfolio, blacklist)
strategy.set_limit(0.3, 0.3)
rebalance = Rebalance(60, long_portfolio, long_factor, blacklist)

backtest = BackTest(long_portfolio, strategy, market, rebalance)
backtest.run()

# print(long_portfolio.value_book)

rebalance on 2022-01-03: [('XLY', -0.009044797322102505), ('VNQ', -0.0009552026778974487)]
rebalance on 2022-03-30: [('XLY', -0.4835865633900216), ('VNQ', -0.01564606643824218), ('XLF', 0.5)]
rebalance on 2022-06-27: [('VNQ', -0.006936033981127354), ('XLF', 0.008826619697691662)]
rebalance on 2022-09-21: [('VNQ', -0.4719801088493264), ('XLF', -0.017675625521206828), ('XLU', 0.49)]
rebalance on 2022-12-15: [('XLF', -0.018083204974613354), ('XLU', 0.017874306792426964)]
rebalance on 2023-03-15: [('XLU', -0.49330713479694493), ('XLF', -0.0060336286736862865), ('XLP', 0.5)]
rebalance on 2023-06-09: [('XLF', -0.5017901634614006), ('XLP', 0.011445342212476561), ('XLV', 0.49)]
rebalance on 2023-09-06: [('XLV', -0.49696062914226236), ('XLP', -0.49301956335835245), ('XTL', 0.49), ('XLK', 0.5)]


In [ ]:
### Short factor
short_factor = SalesGrowthFactor(security_universe, start_date, end_date, "short", 12)
short_position = short_factor.get_position(start_date)
short_portfolio = Portfolio(100.0, start_date, end_date)
short_factor.set_portfolio_at_start(short_portfolio, short_position)

blacklist = []
strategy = StopGainAndLoss(short_portfolio, blacklist)
strategy.set_limit(0.3, 0.3)
rebalance = Rebalance(60, short_portfolio, short_factor, blacklist)

backtest = BackTest(short_portfolio, strategy, market, rebalance)
backtest.run()

# print(short_portfolio.value_book)

rebalance on 2022-01-03: [('XLP', -0.008497149348843935), ('XTL', -0.0015028506511560735)]
rebalance on 2022-03-30: [('XLP', -0.5101929285970792), ('XTL', 0.010698358524716345), ('XLV', 0.5)]
rebalance on 2022-06-27: [('XLV', -0.5194968994700575), ('XTL', 0.03053408056630963), ('XLI', 0.49)]
rebalance on 2022-09-21: [('XTL', -0.49959601664642345), ('XLI', -0.00034022645612141744), ('XLK', 0.5)]


In [ ]:
### plot
import matplotlib.pyplot as plt

%matplotlib inline
benchmark = Benchmark("^SPX", start_date, end_date).get_performance()

metric = Metric(long_portfolio, benchmark)
print(f"portfolio annulized return: {metric.annualized_return()}")
print(
    f"portfolio annulized return relative to benchmark: {metric.annualized_return_relative_to_benchmark()}"
)
print(f"information ratio: {metric.information_ratio()}")

analysis = Analysis(
    long_portfolio,
    short_portfolio,
    benchmark,
    "SPX",
)
analysis.draw()

plt.show()